In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torchvision

import os
import random
import os.path as osp

from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import transforms
import matplotlib.pyplot as plt

In [50]:
LABEL_PATH = "../../data/modified_labels_compass/"
VID_PATH = "../../data/Walking_with_compass/"
PROCESSED_PATH = "../../data/processed"
DATA_SAVE_PATH = "../../data/videos_modified_augmented"
MODELS_PATHS = "./models"
FRAME_RATE = 2

In [51]:
"""
  Preprocess video data.
"""
import subprocess
import cv2

def map_to_multiclass(lab):
    if lab == 'LEFT':
        return 0
    if lab == 'RIGHT':
        return 1
    return 2

def get_all_files_from_dir(directory, basename =None):
    file_paths = []
    # print(directory)
    try:
        for root, dirs, files in os.walk(directory):
            files = [f for f in files if not f[0] == '.']
            if basename is None:
                file_paths += [os.path.join(root, x) for x in files]
            else:
                file_paths += [os.path.join(root, x) for x in files if x.startswith(basename)]
        return sorted(file_paths)
    except Exception as e:
        print("Error: ",e)
    
def get_lab(labels, time):
    for row in labels:
        if time <= float(row[2]) and time >= float(row[1]):
            return row[0]

def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

def inRange(a, b, c, mode = 'full'):
    q = round(a/1000 , 3)
    if(mode == 'full'):
        if (q>=b and q<=c):
            return True
    else:
        if(q>=b):
            return True
    return False

def searchRowConstraint(df_series, a):
    return df_series.searchsorted(a, side='left')

def uniformSampling(lst,vids):
    # [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 
    # 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    if(len(lst)<=20):
        return lst,vids
    m = len(lst)//20
    
    new_lst = []
    new_vids = []
    
    lst.reverse()
    vids.reverse()
    
    for i in range(0,len(lst),m):
        new_lst.append(lst[i])
        new_vids.append(vids[i])
        
    new_lst.reverse()
    new_vids.reverse()
    
    return new_lst, new_vids

def process_video(video_file, label_filepath):
    video_filename = video_file.split('/')[-1].split('.')[0]
    label_files = get_all_files_from_dir(label_filepath, video_filename)
    labels_meta = []
    video_labels = {}
    # video_pos = {}
    total_labels = 0

    for l_files  in label_files:
        print(l_files)
        labels = pd.read_csv(l_files,sep='\t',header=None)
        total_labels += len(labels)
        labels[1] = labels[1]-1
        labels[2] = labels[2]-1
        labels[0] = labels[0].apply(map_to_multiclass)
        l_files = l_files.split('/')[-1].split('.')[0]
        labels[3] = l_files

        pos = labels[0].ne(2).idxmax()
        # print("Pos:",pos)
        pos_start = searchRowConstraint(labels[1], labels.iat[pos,1]-3)
        # print("Pos Start:",pos_start)
        pos_end = searchRowConstraint(labels[1], labels.iat[pos,2]+3)
        # print("Pos Range:",pos_start,", ",pos_end)
        temp = labels[pos_start:pos_end]
        temp[3] = l_files+"_mid"
        labels_meta.append(temp)
        video_labels[l_files+"_mid"] = {"frames":[],"labels":[]}
        video_labels[l_files+"_mid_flip"] = {"frames":[],"labels":[]}
        
        if(pos_end < len(labels)):
            pos_start_right = searchRowConstraint(labels[1], labels.iat[pos_end,2]+2)
            if(pos_start_right < len(labels)):
                pos_end_right = searchRowConstraint(labels[1], labels.iat[pos_start_right,2]+6)
                if(pos_start_right <= pos_end_right - 3): 
                    temp = labels[pos_start_right:pos_end_right]
                    temp[3] = l_files+"_right"
                    labels_meta.append(temp)
                    video_labels[l_files+"_right"] = {"frames":[],"labels":[]}
                    video_labels[l_files+"_right_flip"] = {"frames":[],"labels":[]}

        
        if(pos_start > 0):
            pos_start_left = searchRowConstraint(labels[1], labels.iat[pos_start,1]-7)
            pos_end_left = searchRowConstraint(labels[2], labels.iat[pos_start,2]-1)
            if(pos_start_left <= pos_end_left - 3): 
                temp = labels[pos_start_left:pos_end_left]
                temp[3] = l_files+"_left"
                labels_meta.append(temp)
                video_labels[l_files+"_left"] = {"frames":[],"labels":[]}
                video_labels[l_files+"_left_flip"] = {"frames":[],"labels":[]}
    
    labels = pd.concat(labels_meta)
    labels = labels.sort_values(1)
    labels.reset_index(drop=True,inplace=True)
    
    # labels.to_csv('labels.csv',index=None)
    # return 
    
    
    labels = labels.to_numpy()
    # print(labels)
    # return

    vidcap = cv2.VideoCapture(video_file)
    # fps = vidcap.get(cv2.CAP_PROP_FPS)

    ctr = 0
    lbl = 0
    
    row = labels[lbl]
    hasFrames,image = vidcap.read()
    fl = 0
    
    while(True): 
        
        while (hasFrames and not inRange(vidcap.get(cv2.CAP_PROP_POS_MSEC), float(row[1]), float(row[2]), mode='half')):
            hasFrames,image = vidcap.read()
            # print(vidcap.get(cv2.CAP_PROP_POS_MSEC))

        while(hasFrames and inRange(vidcap.get(cv2.CAP_PROP_POS_MSEC), float(row[1]),float(row[2]))):
            # print("Row: ",lbl)
            try:
                
                save_file_name = row[3] + "_" + str(ctr) + ".npy"
                np.save(osp.join(PROCESSED_PATH, save_file_name), image)
                if(int(row[0]) == 1):
                    fl = 0
                elif(int(row[0]) == 0):
                    fl = 1
                else:
                    fl = 2
                
                save_file_name_flipped = row[3]+"_flip" + "_" + str(ctr) + ".npy"
                np.save(osp.join(PROCESSED_PATH, save_file_name_flipped),  cv2.flip(image, 1))
                
                video_labels[row[3]]['labels'].append(int(row[0]))
                video_labels[row[3]]['frames'].append(save_file_name)
                video_labels[row[3]+"_flip"]['labels'].append(fl)
                video_labels[row[3]+"_flip"]['frames'].append(save_file_name_flipped)
                ctr += 1
                for _ in range(2):
                    hasFrames,image = vidcap.read()

            except Exception as e:
                print(e)
                hasFrames,image = vidcap.read()
            
    
        if(hasFrames == False or lbl >= len(labels)-1):
            print("Has Frames: ", hasFrames)
            break

        lbl += 1
        row = labels[lbl]
   
    frame_count = 0
    for k,v in video_labels.items():
        v['labels'],v['frames'] = uniformSampling(v['labels'],v['frames'])
        # print(len(v['labels']))
        save_file = k + ".csv"
        df = pd.DataFrame(v)
        # print(df.head())
        df.to_csv(osp.join(DATA_SAVE_PATH,save_file), index=None)
        frame_count += len(v['frames'])

    print("After processing:")
    print("Length of labels: ",total_labels)
    print("Labels utilized: ", lbl)
    print("Frames labeled: ", frame_count)
    
def preprocess():
    for video_filename in get_all_files_from_dir(VID_PATH):
        process_video(video_filename, LABEL_PATH)
        print("Finished processing ", video_filename)
        # return


In [52]:
### preprocess videos
preprocess()

../../data/modified_labels_compass/walking_data_1_a.csv
../../data/modified_labels_compass/walking_data_1_b.csv
../../data/modified_labels_compass/walking_data_1_c.csv
../../data/modified_labels_compass/walking_data_1_d.csv
../../data/modified_labels_compass/walking_data_1_e.csv
../../data/modified_labels_compass/walking_data_1_f.csv


/tmp/ipykernel_4077/2400670210.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[3] = l_files+"_mid"
/tmp/ipykernel_4077/2400670210.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[3] = l_files+"_right"
/tmp/ipykernel_4077/2400670210.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Has Frames:  True
After processing:
Length of labels:  405
Labels utilized:  285
Frames labeled:  482
Finished processing  ../../data/Walking_with_compass/walking_data_1.mp4
../../data/modified_labels_compass/walking_data_2_a.csv
../../data/modified_labels_compass/walking_data_2_b.csv
../../data/modified_labels_compass/walking_data_2_c.csv
../../data/modified_labels_compass/walking_data_2_d.csv
../../data/modified_labels_compass/walking_data_2_e.csv
../../data/modified_labels_compass/walking_data_2_f.csv
../../data/modified_labels_compass/walking_data_2_g.csv
../../data/modified_labels_compass/walking_data_2_h.csv
Has Frames:  True
After processing:
Length of labels:  511
Labels utilized:  371
Frames labeled:  588
Finished processing  ../../data/Walking_with_compass/walking_data_2.mp4
../../data/modified_labels_compass/walking_data_3_a.csv
../../data/modified_labels_compass/walking_data_3_b.csv
../../data/modified_labels_compass/walking_data_3_c.csv
../../data/modified_labels_compass/w

In [53]:
BATCH = 1
# SEQUENCE_LENGTH = 20
HEIGHT = 128
WIDTH = 128
CHANNELS = 3

In [61]:
class VideoDataset(Dataset):
    def __init__(self, files, transforms, base_path):
        self.transforms = transforms
        self.files = files
        self.base_path = base_path
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        seq_filename = self.files[idx]
        # print("Data point: ",seq_filename) 
        data = pd.read_csv(seq_filename)
        # print(data.head())
        video = torch.FloatTensor(len(data), CHANNELS, HEIGHT, WIDTH)
        
        for e,filename in enumerate(data['frames']):
            try:
                # print(filename)
                frame = np.load(osp.join(self.base_path,filename), allow_pickle=True)
                frame = (frame - frame.min())/(frame.max() - frame.min())
                frame = self.transforms(frame)

            except Exception as ex:
                print(ex)
                frame = torch.zeros((CHANNELS, HEIGHT, WIDTH))

            video[e,:,:,:] = frame
          
        return video, torch.LongTensor(data['labels'].to_numpy())
        

In [62]:
def make_tt_split(data_folder):
    X = []
    y = []
    n = 0
    for filename in os.listdir(data_folder):
        if(filename[-3:]=="csv"):
            X.append(osp.join(data_folder,filename))
            # df = pd.read_csv(osp.join(data_folder,filename))
            # n += 1
            # for i in range(len(df)-seq_len):
            #     X.append(df['frames'][i:i+seq_len].tolist())
            #     y.append(df['labels'][i:i+seq_len].tolist())
    # print(X)
    random.shuffle(X)
    train_size = int(len(X) * 0.8)
    print("Train size: ",train_size)
    return X[:train_size], X[train_size:]
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # print(X_train)
    # print(n)
    # print(X_test)
    # return X_train, X_test, y_train, y_test



In [63]:
"""
We can also explore https://github.com/okankop/vidaug for video based augmentations.
"""

'\nWe can also explore https://github.com/okankop/vidaug for video based augmentations.\n'

In [64]:
cuda = torch.cuda.is_available()
print(cuda)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# train_transforms = [ttf.ToTensor(), transforms.Resize((HEIGHT, WIDTH)), transforms.ColorJitter(), transforms.RandomRotation(10), transforms.GaussianBlur(3)]
train_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((HEIGHT, WIDTH))])
val_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((HEIGHT, WIDTH))])

X_train, X_test = make_tt_split(DATA_SAVE_PATH)
train_dataset = VideoDataset(files = X_train, transforms=train_transforms,base_path = PROCESSED_PATH)
val_dataset = VideoDataset(files = X_test, transforms=val_transforms, base_path = PROCESSED_PATH)


train_args = dict(shuffle=True, batch_size=BATCH, num_workers=2, pin_memory=True, drop_last=False) if cuda else dict(shuffle=True, batch_size=BATCH, drop_last=False)
train_loader = DataLoader(train_dataset, **train_args)

val_args = dict(shuffle=False, batch_size=BATCH, num_workers=2, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=BATCH, drop_last=False)
val_loader = DataLoader(val_dataset, **val_args)



True
Train size:  144


In [65]:
print(len(train_dataset))
print(len(val_dataset))

144
36


In [66]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):

    """
    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.
    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """
        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h) #[batch_size, self.hidden_dim, height, width]

            layer_output = torch.stack(output_inner, dim=1) #[batch_size,t,self.hidden_dim, height, width]
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [67]:
class ConvLSTMModel(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False, num_classes = 3):
        super(ConvLSTMModel, self).__init__()
        self.convlstm = ConvLSTM(input_dim, hidden_dim, kernel_size, num_layers,batch_first, bias, return_all_layers)
        self.linear = nn.Linear(hidden_dim * HEIGHT * WIDTH, num_classes)

    def forward(self, input_tensor, hidden_state=None):
      x,_ = self.convlstm(input_tensor)
      # print(x[0].shape)  # torch.Size([2, 8, 128, 256, 256])
      x = torch.flatten(x[0], start_dim=2)
      # print(x.shape)  	# torch.Size([2, 8, 8388608])
      x = self.linear(x) #op: [batch, t, num_classes]
      return x


In [77]:
def save(model, index, optim = False):
    if not os.path.exists(MODELS_PATHS+'/attempt5_1sec_prior_labels_modified_data'):
        os.mkdir(MODELS_PATHS+'/attempt5_1sec_prior_labels_modified_data')
    if(optim):
        torch.save(model.state_dict(), MODELS_PATHS+'/attempt5_1sec_prior_labels_modified_data'+'/optimizer_params_{:08d}.pth'.format(index))
    else:
        torch.save(model.state_dict(), MODELS_PATHS+'/attempt5_1sec_prior_labels_modified_data'+'/model_params_{:08d}.pth'.format(index))

In [106]:
lr = 0.0005 #changed from 0.01
epochs = 50
lamda = 1e-2  #L2 regularization #changed from 1e-4
num_classes = 3
convlstm_hidden = 128
num_conv_lstm_layers = 2

model = ConvLSTMModel(CHANNELS,convlstm_hidden,(3,3),num_conv_lstm_layers,True)
model.load_state_dict(torch.load('./models/attempt5_1sec_prior_labels_modified_data/model_params_00000044.pth'))  #Validation: 75.4808%
model = model.to(device)

# class_weights = [2,2,1] 
# class_weights = torch.Tensor(class_weights)
# class_weights = class_weights.to(device)

# criterion = nn.CrossEntropyLoss(weight =class_weights)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=lamda, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=lamda)
optimizer.load_state_dict(torch.load('./models/attempt5_1sec_prior_labels_modified_data/optimizer_params_00000044.pth')) #Validation: 75.4808%

for g in optimizer.param_groups:
    g['lr'] = lr
    g['weight_decay']= lamda
    
scaler = torch.cuda.amp.GradScaler()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
print(model)

ConvLSTMModel(
  (convlstm): ConvLSTM(
    (cell_list): ModuleList(
      (0): ConvLSTMCell(
        (conv): Conv2d(131, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): ConvLSTMCell(
        (conv): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
  )
  (linear): Linear(in_features=2097152, out_features=3, bias=True)
)


In [107]:
for epoch in range(50,50+epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    num_correct = 0
    total_loss = 0
    
    frames_count = 0
    
    for i, (x, y) in enumerate(train_loader):
        # print(x.shape)
        frames_count += x.shape[1]
        model.train()
        optimizer.zero_grad()

        x = x.float().to(device)
        y = y.to(device)
        
        with torch.cuda.amp.autocast():
            outputs = model(x)
            del x
            loss = criterion(outputs.view(-1,num_classes), y.long().view(-1))

        num_correct += int((torch.argmax(outputs, axis=2) == y).sum())
        del outputs
        total_loss += float(loss)

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / frames_count),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        scaler.scale(loss).backward()
        scaler.step(optimizer) 
        scaler.update()

        scheduler.step()

        batch_bar.update() # Update tqdm bar
        

    batch_bar.close()

    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        100 * num_correct / frames_count,
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    
    save(model, epoch)
    save(optimizer, epoch, optim=True)
    
    # validation
    model.eval()
    val_num_correct = 0
    frames_count = 0
    
    for i, (vx, vy) in enumerate(val_loader):
      
        vx = vx.to(device)
        vy = vy.to(device)
        frames_count += vx.shape[1]
        
        with torch.no_grad():
            outputs = model(vx)
            del vx

        val_num_correct += int((torch.argmax(outputs, axis=2) == vy).sum())
        del outputs

    print("Validation: {:.04f}%".format(100 * val_num_correct / frames_count))

    
batch_bar.close()

Epoch 51/50: Train Acc 87.7129%, Train Loss 5.3651, Learning Rate 0.0005
Validation: 70.6731%


Epoch 52/50: Train Acc 92.3662%, Train Loss 3.2925, Learning Rate 0.0005
Validation: 77.4038%


Epoch 53/50: Train Acc 93.1873%, Train Loss 2.6006, Learning Rate 0.0005
Validation: 71.9952%


Epoch 54/50: Train Acc 94.1910%, Train Loss 2.0015, Learning Rate 0.0005
Validation: 64.1827%


Epoch 55/50: Train Acc 92.5182%, Train Loss 2.3271, Learning Rate 0.0005
Validation: 62.7404%


...

Epoch 68/50: Train Acc 93.8564%, Train Loss 1.6828, Learning Rate 0.0004
Validation: 71.2740%


Epoch 69/50: Train Acc 94.8905%, Train Loss 1.1969, Learning Rate 0.0003
Validation: 68.9904%


Epoch 70/50: Train Acc 87.2567%, Train Loss 6.5776, Learning Rate 0.0003
Validation: 47.2356%


Train:  74%|████████████████████████▌        | 107/144 [01:12<00:25,  1.43it/s, acc=92.1689%, loss=2.3730, lr=0.0003, num_correct=2248]

KeyboardInterrupt: 

Validation: 65.3846%
